In [53]:
import findspark
findspark.init()
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [54]:
import os
tDf = spark\
    .read\
    .options(header='false', inferschema='true', delimiter='\t')\
    .csv(os.path.join('data', 'ds_spark_heightweight.txt'))

# 컬럼 조회 select

In [55]:
myDf = spark\
        .read.format('com.databricks.spark.csv')\
        .options(header='true', inferschema='true')\
        .load(os.path.join('data','myDf.csv'))

In [56]:
_name=myDf.select('name')
_name.show()

+----+
|name|
+----+
| js1|
| js2|
+----+



In [57]:
_name=myDf.select('name', 'age').show()

+----+---+
|name|age|
+----+---+
| js1| 10|
| js2| 20|
+----+---+



In [58]:
cols = ['name', 'age']
myDf.select(*cols).show()

+----+---+
|name|age|
+----+---+
| js1| 10|
| js2| 20|
+----+---+



# 컬럼을 list로 변환

In [59]:
myDf.select('name').collect()

[Row(name='js1'), Row(name='js2')]

In [60]:
myDf.select('name').rdd.map(lambda x: x[0]).collect()

['js1', 'js2']

In [61]:
myDf.select('name').rdd.flatMap(lambda x: x).collect()

['js1', 'js2']

In [62]:
myDf.select("name", "age", myDf.name.like("js,%")).show()

+----+---+--------------+
|name|age|name LIKE js,%|
+----+---+--------------+
| js1| 10|         false|
| js2| 20|         false|
+----+---+--------------+



In [63]:
myDf.select("name", "age", myDf.name.startswith("j")).show()

+----+---+-------------------+
|name|age|startswith(name, j)|
+----+---+-------------------+
| js1| 10|               true|
| js2| 20|               true|
+----+---+-------------------+



In [64]:
myDf.select("name", "age", myDf.name.endswith("lee")).show()

+----+---+-------------------+
|name|age|endswith(name, lee)|
+----+---+-------------------+
| js1| 10|              false|
| js2| 20|              false|
+----+---+-------------------+



In [65]:
myDf1 = myDf.alias("myDf1")

In [66]:
myDf1.select(myDf1.name.substr(1,3).alias("short name")).show(3)

+----------+
|short name|
+----------+
|       js1|
|       js2|
+----------+



In [67]:
from pyspark.sql.functions import when
myDf.select("age", when(myDf.age < 11, 1).otherwise(0)).show()

+---+--------------------------------------+
|age|CASE WHEN (age < 11) THEN 1 ELSE 0 END|
+---+--------------------------------------+
| 10|                                     1|
| 20|                                     0|
+---+--------------------------------------+



In [68]:
from pyspark.sql.functions import when
myDf.select("age", (when(myDf.age < 11, 1).otherwise(0)).alias('<11')).show()

+---+---+
|age|<11|
+---+---+
| 10|  1|
| 20|  0|
+---+---+



In [69]:
from pyspark.sql.functions import when
_myDf=myDf.select(when(myDf['age'] >11.0, ">11").otherwise("<11").alias("how tall")).show()

+--------+
|how tall|
+--------+
|     <11|
|     >11|
+--------+



In [70]:
myDf.where(myDf['age'] < 11).show()

+---+----+---+
|_c0|name|age|
+---+----+---+
|  0| js1| 10|
+---+----+---+



In [71]:
myDf.where(myDf['age'] < 11)\
    .select(myDf['name'], myDf['age']).show()

+----+---+
|name|age|
+----+---+
| js1| 10|
+----+---+



# filter

In [72]:
myDf.filter(myDf['age'] > 11).show()

+---+----+---+
|_c0|name|age|
+---+----+---+
|  1| js2| 20|
+---+----+---+



# regexp_replace 컬럼의 내용 변경

In [73]:
from pyspark.sql.functions import *

_heightDf = myDf.withColumn('nameNew', regexp_replace('name', 'js1', 'lim'))
_heightDf.show()

+---+----+---+-------+
|_c0|name|age|nameNew|
+---+----+---+-------+
|  0| js1| 10|    lim|
|  1| js2| 20|    js2|
+---+----+---+-------+



# groupBy

In [74]:
myDf.groupby(myDf['age'])

In [75]:
myDf.groupby(myDf['age']).max().show()

+---+--------+--------+
|age|max(_c0)|max(age)|
+---+--------+--------+
| 20|       1|      20|
| 10|       0|      10|
+---+--------+--------+



In [80]:
myDf.groupBy('age').agg({"age":"sum"}).show()

+---+--------+
|age|sum(age)|
+---+--------+
| 20|      20|
| 10|      10|
+---+--------+



In [78]:
wcDf.groupBy(wcDf.ClubCountry).count().show()


NameError: name 'wcDf' is not defined